<a href="https://colab.research.google.com/github/ISE-CS4445-AI/CS4445-AI-Practice/Week-5_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 Exercise Notebook: 
## Practical RNN Implementation on IMDB Sentiment Analysis with PyTorch

In this notebook, we will:
- Load the IMDB dataset (movie reviews labeled as positive/negative).
- Preprocess the text data (tokenization, vocabulary building, numericalization, and padding).
- Build a simple RNN (using an LSTM cell) for sentiment classification.
- Train the model on a small subset (for demonstration) and evaluate predictions.
- Use the trained model to make a fun prediction on a custom review.

> **Note:** Training on the full IMDB dataset will take time. For demonstration, we are going to use a subset of randomly sampled 1000 datapoints.

---
## Importing Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import random

# Set random seeds for reproducibility
torch.manual_seed(42)
random.seed(42)

## Loading and exploring the IMDB dataset

In [ ]:
# Download the IMDB dataset using torchtext.
# Each example is a tuple: (label, text)
train_iter = list(IMDB(split='train'))
test_iter = list(IMDB(split='test'))

# For quick demonstration, we will use a small subset of the training data.
# You can adjust the subset size if needed.
subset_size = 1000
train_data = random.sample(train_iter, subset_size)

print(f"Total training examples (subset): {len(train_data)}")
print("Example:", train_data[0])